# Severity Index Assessment for the Southwest United States during Hurricane Season
### Authors: Ellie Rafalski and Landon Allmaras
### Contacts: lda5159@psu.edu and efr5356@psu.edu

The figure below is a sample output of our applied **Severity Index** and **Normalized Components** from the August 5th event we chose as one of our analysis events.

![image](ms3-1.PNG)
![image2](ms3-2.PNG)

#### This project presents a potential methodology for calculating a 'Damage Severity Index'. We have gathered a few different ECMWF model runs that range from a low risk tropical storm to a hurricane that can cause major damage. Along with that, bouncing between different variables that include wind speed, surface pressure, precipitation can prove helpful to emergency planners, utility managers, and insurers in assessing the potential impact of hurricanes and tropical storms.

## How may a potential employer use this?
1. Track the progression of tropical systems and highlight areas where conditions may be worsening.
2. Indicate potential trends in storm severity that could lead to greater impacts in specific regions.
3. Provide general guidance that could support public awareness around evacuation and return timing.
4. Offer a preliminary reference that may assist in post-storm impact assessments and insurance evaluations.
5. Flag areas of elevated risk that could help inform initial utility restoration priorities.

![damage](damage473.jpg)

## **Choosing which data to use and how to navigate downloading it:**

In [ ]:
from herbie import Herbie, FastHerbie
import pandas as pd, numpy as np
import xarray as xr
import dask

runtime = pd.Timestamp("2024-09-27", tz="utc").replace(tzinfo=None).floor('24h')
H = Herbie(runtime, model="ifs", product="oper", fxx=6, save_dir='.', overwrite=True)

- Using Herbie is a helpful tool when trying to pull any old runs that you may be interested in
- This is also a single run - in this case it will start at 00 Z and take the 6 hour forecast run
- Out of this single run we defined which variables we wanted to use that we believed could be helpful for putting together our **Severity Index**

In [ ]:
runtime = pd.Timestamp("2024-09-27", tz="utc").replace(tzinfo=None).floor('24h')
H = FastHerbie([runtime], model="ifs", product="oper", fxx=np.arange(0,246,6).tolist(), save_dir='.', overwrite=True)

- We also included capability of handling full model runs, by incorporating Fast Herbie
- Similar ideas of pulling out defined variables that were helpful in building our **Severity Index**
- We chose three different tropical events to test this index on, Tropical Storm Arlene, Hurricane Helene, and Hurricane Debby (with ranging severity)

### Vorticity, Surface Pressure, Total Wind, and Precipitation rate
- Defining graphs for each of our variables gave us an idea of what could be a crucial part of the calculation and watching the progress of the run.
- We also combined a few different contour and contourf values to read different components on the projection.

Image of combined wind components and temperature:
![image4](ms3-4.png)
Image of surface pressure and precipitation:
![image5](ms3-5.png)

## Following our download of a full model run... we defined our equation for the Severity index:

In [ ]:
severity_index = abs((0.5 * totalwind_norm) + (0.3 * prate_norm) + (0.15 * vorticity_norm) + (0.05 * pressure_norm))
severity_index = severity_index * 10

- To understand and track the severity of tropical weather systems, the index uses four important weather variables: wind speed, precipitation rate, vorticity, and surface pressure. Each of these factors helps us learn more about the storm's behavior and its potential impacts.
- After selecting and pulling out this data from our downloaded netCDF files, we then normalized the data to plug into our index equation.
- Wind Speed: Strong winds can cause significant damage, including destruction of buildings, fallen trees, and power outages. By including wind speed in the index, we can identify areas that might be most affected by wind-related impacts.
- Precipitation Rate: Intense rainfall can lead to flash flooding and strain drainage systems. Areas with higher precipitation rates are often at greater risk for flooding, which is a major concern during tropical events.
- Vorticity: Vorticity measures the spin of the air in the storm, helping us understand the storm’s organization and intensity. Higher vorticity often indicates a more organized storm, which is linked to stronger winds and potentially more damage.
- Surface Pressure: Low surface pressure is a key feature of tropical storms, and when pressure drops, it typically means the storm is strengthening. Monitoring surface pressure helps us track how the storm is evolving over time.
- These four variables together provide a more complete picture of the storm’s severity, helping us understand both the immediate and longer-term risks as the storm progresses.



**Explaining the Index**

- The severity index is designed to provide a clear indication of the potential impacts of a tropical system. To make the index more interpretable, it is scaled from 0 to 5, where:

- 0 represents minimal or no expected damage, indicating conditions that are not severe.

- 5 represents the highest severity, indicating conditions that could lead to widespread damage and danger.

The scale was determined based on an assessment of the key meteorological variables—wind speed, precipitation rate, vorticity, and surface pressure—and how they correspond to known impacts in tropical weather events. By normalizing these values and scaling them to a 0-5 range, we can quantify the potential for storm-related impacts in a way that is both understandable and practical for further analysis and decision-making.

## Calculation of the Index

Development of multiple functions allowed us to take any ECMWF model data as input, and output all of the normalized variables and the calculated **Severity Index**

In [ ]:
def normalize(var, min_val, max_val): 
    return (var - min_val) / (max_val - min_val)

def find_vorticity(lst):
    vo = lst['vo'].sel(isobaricInhPa=500) 
    vorticity_norm = normalize(vo, -0.00001, 0.003)
    return vorticity_norm

# finding the pressure normalization 
def find_pressure(lst):
    sp = lst['sp'].values / 100
    pressure_norm = normalize(sp, 980, 1020)
    return pressure_norm

# finding the precipitation rate normalization
def find_prate(lst):
    tp = lst['tp'].values
    approximatedprate = (tp * 39.37) / 6  # conversion to inches per hour (originally was in meters)
    prate_norm = normalize(approximatedprate, 0.1, 1.0)
    return prate_norm

# find wind normalization
def find_wind(lst):
    uNew = lst['u'].sel(isobaricInhPa=1000).values
    vNew = lst['v'].sel(isobaricInhPa=1000).values
    totalwind = ((uNew ** 2 + vNew ** 2) ** 0.5) * 2 # conversion to knots for easier plotting
    totalwind_norm = normalize(totalwind, 0, 140)
    return totalwind_norm
    
def find_threat(lst):
     # pulling in the previously defined functions and use them on the inputted list
    pressure_norm = find_pressure(lst)
    vorticity_norm = find_vorticity(lst)
    totalwind_norm = find_wind(lst)
    prate_norm = find_prate(lst)

    # threat formula 
    severity_index = abs((0.5 * totalwind_norm) + (0.3 * prate_norm) + (0.15 * vorticity_norm) + (0.05 * pressure_norm))
    # weighted severity index, we decided to have it mostly based on total wind and precipitation, with small components of the vorticity and pressure for some more depth
    severity_index = severity_index * 10
    # multitplied our index by ten to make the values a little more " realistic " to have a range 1-5 instead of decimals
    return severity_index

Now that we introduced our Severity Index, we can make subplots for our variables that were chosen and include the movement of the storm and where it is going to be affected at the most

## Example Application

This image correlates with Hurricane Helene at 06 Z
![image6](ms3-6.png)
![image7](ms3-7.png)

This plot shows the severity index applied to Hurricane Helene on 09-27-2024 at forecast hour 6. The event was chosen because it featured notable tropical weather impacts, making it a good test case for evaluating how the index responds to real conditions.

In this example, the index highlights areas of increased severity where strong winds, heavy rainfall, and low surface pressure were present. These regions correspond well with the observed core of the storm, showing that the index can capture areas of potential concern during a tropical event. While the index is still a simplified representation, it helps visualize changing conditions and can offer early insight into where storm impacts may be highest.

## Conclusion
This project introduced a basic severity index for tropical weather systems, combining wind speed, precipitation rate, vorticity, and surface pressure into a single 0–5 scale. While the index is still in its early stages, it has shown potential in identifying areas of concern during tropical events.

Future improvements could include incorporating additional factors like storm surge, terrain, and population density to better assess real-world impacts. Enhancing the index's accuracy and expanding its applicability could make it a more valuable tool for forecasting and decision-making in meteorology.